In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot

In [ ]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.utils.vis_utils import plot_model
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
import sklearn.metrics as metrics
from sklearn.metrics.classification import accuracy_score
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, LSTM
#from dbn.tensorflow import SupervisedDBNClassification
import h5py
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
from sklearn.model_selection import KFold,cross_val_score
import matplotlib.pyplot as plt
import scikitplot as skplt
kfold=KFold(5,True,3)
cvscores = []

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#### DNN #######
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/Code/dataset/IoT-BoT.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
data = cleaningdata(dataset)

# collecting all required colomns in data to X
X = data.iloc[:, 0:85].values

# collecting our LABEL colomn in y
y = data.iloc[:, -2].values

In [ ]:
from sklearn.preprocessing import label_binarize
Y = label_binarize(y, classes=[0, 1, 2, 3, 4])
n_classes = Y.shape[1]
print(n_classes)

for train, test in kfold.split(X,y):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=52)
   from keras.utils.np_utils import to_categorical
   sc = StandardScaler()
   X_train = sc.fit_transform(X_train)
   X_test = sc.transform(X_test)

   #X_train.shape

   X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
   X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

   x_train = np.array(X_train)
   x_test = np.array(X_test)
   y_train1 = np.array(y_train)
   y_test1 = np.array(y_test)

   y_train= to_categorical(y_train1)
   y_test= to_categorical(y_test1)
# 1. define the network
   model = Sequential()
   model.add(Dense(800, input_dim=85, activation='relu'))
   model.add(Dropout(0.8))
   model.add(Dense(400, activation='relu'))
   model.add(Dropout(0.8))
   model.add(Dense(5,activation='softmax'))
   model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
   history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
   scores = model.evaluate(X_test, y_test, verbose=1)
   print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
   cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


5
Epoch 1/10
24255/24255 [==============================] - 208s 9ms/step - loss: 0.0457 - accuracy: 0.9954 - mae: 0.0025
Epoch 2/10
24255/24255 [==============================] - 208s 9ms/step - loss: 0.0338 - accuracy: 0.9986 - mae: 6.0917e-04
Epoch 3/10
24255/24255 [==============================] - 207s 9ms/step - loss: 0.0682 - accuracy: 0.9989 - mae: 4.5863e-04
Epoch 4/10
24255/24255 [==============================] - 204s 8ms/step - loss: 0.0630 - accuracy: 0.9991 - mae: 3.7177e-04
Epoch 5/10
24255/24255 [==============================] - 210s 9ms/step - loss: 0.0530 - accuracy: 0.9992 - mae: 3.2800e-04
Epoch 6/10
24255/24255 [==============================] - 200s 8ms/step - loss: 0.0536 - accuracy: 0.9992 - mae: 3.1930e-04
Epoch 7/10
24255/24255 [==============================] - 206s 8ms/step - loss: 0.0577 - accuracy: 0.9993 - mae: 2.7861e-04
Epoch 8/10
24255/24255 [==============================] - 203s 8ms/step - loss: 0.0613 - accuracy: 0.9993 - mae: 2.9856e-04
Epoch 9/10

In [ ]:
target_names = ['DoS', 'DDOS', 'Reconnaissance', 'Normal','Theft']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
#print("Cohens kappa:",metrics.cohen_kappa_score(expected, predicted))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    508847
          DDOS       1.00      1.00      1.00    521139
Reconnaissance       1.00      1.00      1.00     77830
        Normal       1.00      1.00      1.00    444081
         Theft       0.95      0.94      0.95       414

     micro avg       1.00      1.00      1.00   1552311
     macro avg       0.99      0.99      0.99   1552311
  weighted avg       1.00      1.00      1.00   1552311
   samples avg       1.00      1.00      1.00   1552311

Accuracy_Train: 0.9998499012118062
Precision_Train: 0.9998505547916599
Recall_Train: 0.9998499012118062
f1-score_Train: 0.9998502094965062
ROC_AUC_Train: 0.9999967264205893


In [ ]:
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Test:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))


                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    127692
          DDOS       1.00      1.00      1.00    129983
Reconnaissance       1.00      1.00      1.00     19367
        Normal       1.00      1.00      1.00    110930
         Theft       0.92      0.92      0.92       106

     micro avg       1.00      1.00      1.00    388078
     macro avg       0.98      0.98      0.98    388078
  weighted avg       1.00      1.00      1.00    388078
   samples avg       1.00      1.00      1.00    388078

Accuracy: 0.9998041630806178
Precision: 0.9998042437040416
Recall: 0.9998041630806178
f1-score: 0.99980417217909
ROC_AUC_Test: 0.9999961792384175
